In [69]:
import numpy as np
import imutils
import cv2

In [70]:
def detectAndDescribe(image):
    descriptor=cv2.SIFT_create()
    kps,features=descriptor.detectAndCompute(image,None)
    kps=np.float32([kp.pt for kp in kps])
    return (kps,features)

In [71]:
def matchInterestPoints(interestA,interestB,xA,xB,ratio,re_proj):
    matcher=cv2.BFMatcher()
    rawMatches=matcher.knnMatch(xA,xB,2)
    matches=[]
    for m in rawMatches:
        if len(m) == 2 and m[0].distance<m[1].distance*ratio:
            matches.append((m[0].trainIdx,m[0].queryIdx))
    if len(matches)>4:
        ptsA=np.float32([interestA[i] for (_,i) in matches])
        ptsB=np.float32([interestB[i] for (i,_) in matches])
        H,status=cv2.findHomography(ptsA,ptsB,cv2.RANSAC,re_proj)
        return (matches,H,status)
    return None

In [72]:
def drawMatches(imageA,imageB,interestA,interestB,matches,status):
    hA,wA=imageA.shape[:2]
    hB,wB=imageB.shape[:2]
    viz=np.zeros((max(hA,hB),wA+wB,3),dtype="uint8")
    viz[0:hA,0:wA]=imageA
    viz[0:hB,wA:]=imageB
    for((trainIdx,queryIdx),s) in zip(matches,status):
        if s == 1:
            ptA=(int(interestA[queryIdx][0]),int(interestA[queryIdx][1]))
            ptB=(int(interestB[trainIdx][0])+wA,int(interestB[trainIdx][1]))
            cv2.line(viz,ptA,ptB,(0,255,0),1)
    return viz

In [73]:
def stitch(images,ratio=0.75,re_proj=5.0,show_overlay=False):
    imageB,imageA=images
    interestA,xA=detectAndDescribe(imageA)
    interestB,xB=detectAndDescribe(imageB)
    M=matchInterestPoints(interestA,interestB,xA,xB,ratio,re_proj)
    if M is None:
        print("Not enough matches found.")
        return None
    matches,H,status=M
    pano_img=cv2.warpPerspective(imageA,H,(imageA.shape[1]+imageB.shape[1],imageA.shape[0]))
    pano_img[0:imageB.shape[0],0:imageB.shape[1]]=imageB
    if show_overlay:
        visualization=drawMatches(imageA,imageB,interestA,interestB,matches,status)
        return (pano_img,visualization)
    return pano_img

In [74]:
imageA=cv2.imread("left2.jpg")
imageB=cv2.imread("right2.jpg")
imageA=imutils.resize(imageA,width=600)
imageB=imutils.resize(imageB,width=600)

In [75]:
def show(img,time=3,msg="Image"):
    cv2.imshow(msg,img)
    cv2.waitKey(int(time*1000))
    cv2.destroyAllWindows()

In [76]:
pano_img,viz=stitch([imageA,imageB],show_overlay=True)
# show(imageA,3,"Image A")
# show(imageB,3,"Image B")
show(viz,15,"Matched Interest Points")
cv2.imwrite("InterestPoints2.jpg",viz)
show(pano_img,3,"Stitched Image")
cv2.imwrite("stitched2.jpg",pano_img)

True